In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
settings

{'database.directory': '/home/khalil.nor/Code/RMG-database/input',
 'test_data.directory': '/home/khalil.nor/Code/RMG-Py/rmgpy/test_data'}

In [ ]:
#these families just had training rxns added to them 

#dict_keys(['1,2_Insertion_carbene', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement'])

In [3]:
thermo_libs = [
'C1_C2_Fluorine', #adding Siddha's as first most trusted because this is the thermo library that Franklin used
'NCSU_C2_C8_PFAS', #adding Westmoreland's thermo as the second most trusted
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

kin_families = ['1,2_Insertion_carbene', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement']

In [4]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = kin_families,
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


In [5]:
database.kinetics.families

{'1+2_Cycloaddition': <ReactionFamily "1+2_Cycloaddition">,
 '1,2_Insertion_CO': <ReactionFamily "1,2_Insertion_CO">,
 '1,2_Insertion_carbene': <ReactionFamily "1,2_Insertion_carbene">,
 '1,3_Insertion_CO2': <ReactionFamily "1,3_Insertion_CO2">,
 '1,3_sigmatropic_rearrangement': <ReactionFamily "1,3_sigmatropic_rearrangement">,
 'R_Addition_MultipleBond': <ReactionFamily "R_Addition_MultipleBond">,
 'Singlet_Carbene_Intra_Disproportionation': <ReactionFamily "Singlet_Carbene_Intra_Disproportionation">,
 'XY_Addition_MultipleBond': <ReactionFamily "XY_Addition_MultipleBond">}

In [6]:
# '1,3_sigmatropic_rearrangement' 
family_to_train = "1,3_sigmatropic_rearrangement"
family = database.kinetics.families[family_to_train]

In [7]:
family.clean_tree()

In [8]:
start = time.time()
family.generate_tree(thermo_database=database.thermo,
                     nprocs=1,
                     new_fraction_threshold_to_reopt_node=0.25,
                     max_batch_size=800,
                     extension_iter_max=2,
                     extension_iter_item_cap=100)

end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([181.425,211.751,232.143,247.468,271.725,290.011,303.982],'J/(mol*K)'), H298=(-1584.09,'kJ/mol'), S298=(751.842,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(295.164,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-CsCs) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsJ2_singlet-CsH) + group(COFFO)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([181.425,211.751,232.143,247.468,271.725,290.011,303.982],'J/(mol*K)'), H298=(-1584.09,'kJ/mol'), S298=(751.842,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(295.164,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-CsCs) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsC

165.70419049263


In [9]:
len(family.groups.entries)

56

In [10]:
family.groups.entries

{'Root': <Entry index=0 label="Root">,
 'Root_1R!H->C': <Entry index=1 label="Root_1R!H->C">,
 'Root_N-1R!H->C': <Entry index=2 label="Root_N-1R!H->C">,
 'Root_1R!H->C_1C-inRing': <Entry index=3 label="Root_1R!H->C_1C-inRing">,
 'Root_1R!H->C_N-1C-inRing': <Entry index=4 label="Root_1R!H->C_N-1C-inRing">,
 'Root_N-1R!H->C_Ext-4R!H-R_Ext-4R!H-R_Ext-3R!H-R': <Entry index=5 label="Root_N-1R!H->C_Ext-4R!H-R_Ext-4R!H-R_Ext-3R!H-R">,
 'Root_N-1R!H->C_Ext-4R!H-R_5R!H->C': <Entry index=6 label="Root_N-1R!H->C_Ext-4R!H-R_5R!H->C">,
 'Root_N-1R!H->C_Ext-4R!H-R_N-5R!H->C': <Entry index=7 label="Root_N-1R!H->C_Ext-4R!H-R_N-5R!H->C">,
 'Root_1R!H->C_1C-inRing_2R!H->N': <Entry index=8 label="Root_1R!H->C_1C-inRing_2R!H->N">,
 'Root_1R!H->C_1C-inRing_N-2R!H->N': <Entry index=9 label="Root_1R!H->C_1C-inRing_N-2R!H->N">,
 'Root_1R!H->C_N-1C-inRing_Ext-4R!H-R': <Entry index=10 label="Root_1R!H->C_N-1C-inRing_Ext-4R!H-R">,
 'Root_1R!H->C_N-1C-inRing_Ext-1C-R': <Entry index=11 label="Root_1R!H->C_N-1C-inR

In [11]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.0062274932861328125


In [12]:
start = time.time()
family.regularize(thermo_database=database.thermo)
end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([181.425,211.751,232.143,247.468,271.725,290.011,303.982],'J/(mol*K)'), H298=(-1584.09,'kJ/mol'), S298=(751.842,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(295.164,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-CsCs) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsJ2_singlet-CsH) + group(COFFO)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([181.425,211.751,232.143,247.468,271.725,290.011,303.982],'J/(mol*K)'), H298=(-1584.09,'kJ/mol'), S298=(751.842,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(295.164,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-CsCs) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsC

1.5958256721496582


In [13]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([181.425,211.751,232.143,247.468,271.725,290.011,303.982],'J/(mol*K)'), H298=(-1584.09,'kJ/mol'), S298=(751.842,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(295.164,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-CsCs) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsJ2_singlet-CsH) + group(COFFO)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([181.425,211.751,232.143,247.468,271.725,290.011,303.982],'J/(mol*K)'), H298=(-1584.09,'kJ/mol'), S298=(751.842,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(295.164,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-CsCs) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsC

1.520122766494751


In [14]:
len(templateRxnMap)

56

In [15]:
family.clean_tree_rules()

In [16]:
start = time.time()
family.make_bm_rules_from_template_rxn_map(templateRxnMap)#,nprocs=6)
end = time.time()
print(end-start)

/home/khalil.nor/Code/RMG-Py/rmgpy/data/kinetics/family.py:3620: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for rxns, label in rxnlists])
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


1.9915597438812256


In [17]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.009530305862426758


In [18]:
start = time.time()
errors,uncertainties = family.cross_validate(iters=0,random_state=5,folds=0,ascend=False)
end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([181.425,211.751,232.143,247.468,271.725,290.011,303.982],'J/(mol*K)'), H298=(-1584.09,'kJ/mol'), S298=(751.842,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(295.164,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-CsCs) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsJ2_singlet-CsH) + group(COFFO)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([181.425,211.751,232.143,247.468,271.725,290.011,303.982],'J/(mol*K)'), H298=(-1584.09,'kJ/mol'), S298=(751.842,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(295.164,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-CsCs) + group(CsC2sFFF) + longDistanceInteraction_noncyclic(Cs(F)3-R-Cs(F)3) + group(CsC

1.8798112869262695


/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [19]:
save_path = os.path.join(settings['database.directory'], 'kinetics', 'families', family.name)
print(save_path)

/home/khalil.nor/Code/RMG-database/input/kinetics/families/1,3_sigmatropic_rearrangement


In [20]:
family.save(save_path)

In [ ]:
#ran to completion on Apr 30th, 2:45 pm 

In [ ]:
plt.figure(figsize=(10,8))
plt.hist(np.abs(list(errors.values())),bins=30, density=True)
plt.title('Decision Tree Estimator',fontsize=18)
#plt.ylim(0,.4)
plt.xlim(0,15)
plt.ylabel('Probability density',fontsize=18)
plt.xlabel(r'$|Ln(k_{est}/k_{rxn})|$',fontsize=18)